In [137]:
import sys
import datetime
import urllib.request
import time
import json
import os
import io
import pandas
import numpy as np
import multiprocessing as mp
import threading_jobs as tj

from bokeh.core.properties import value
from bokeh.io import show, output_file, export_svgs
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.transform import dodge
from bokeh.io import export_svgs

In [140]:
!conda install --yes --prefix {sys.prefix} -c bokeh selenium

Solving environment: ...working... 
  - anaconda::conda-4.7.12-py37_0
  - defaults::conda-4.7.12-py37_0done

## Package Plan ##

  environment location: D:\Anaconda3

  added / updated specs:
    - selenium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    selenium-3.141.0           |   py37he774522_0         805 KB
    ------------------------------------------------------------
                                           Total:         805 KB

The following NEW packages will be INSTALLED:

  selenium           pkgs/main/win-64::selenium-3.141.0-py37he774522_0




selenium-3.141.0     | 805 KB    |            |   0% 
selenium-3.141.0     | 805 KB    | 1          |   2% 
selenium-3.141.0     | 805 KB    | ########## | 100% 
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


In [38]:
years = ["2010", "2011", "2012", "2013", "2014"]
subreddits = ["nyc", "losangeles", "chicago", "houston", "phoenix"]

# collection of all data aggregated together
reddit_master_collection = pandas.read_csv("D:\social_media_analytics\\reddit_content\collection\sentiment_data_2010-2014.csv", usecols=[1,2,3,4,5,6])


In [39]:
# date from epoch
def getDateFromEpoch(epoch):
    return datetime.datetime.utcfromtimestamp(epoch)


# convert all of our dataframe rows to have date (year-month-day) instead of epoch
for i, row in reddit_master_collection.iterrows():
    date = None
    date = getDateFromEpoch(reddit_master_collection.at[i, 'created_utc'])
    reddit_master_collection.at[i, 'created_utc'] = date
    if i % 20000 == 0:
        print('Iteration', i, end='\r')
        sys.stdout.flush()

In [40]:
# creating a base object incase I accidentally destroy the base object
# i don't want to have to wait to process a new dataframe if I accidentally destroy it

reddit_master_collection2 = reddit_master_collection
# reddit_master_collection = reddit_master_collection2

In [42]:
# remove excess records

starting_datetime = datetime.datetime.strptime('2010-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')
reddit_master_collection = reddit_master_collection[reddit_master_collection.created_utc >= starting_datetime]


In [43]:
# turn them into year-month-day
for i, row in reddit_master_collection.iterrows():
    #datetime.datetime.strptime(when, '%Y-%m-%d').date()
    date = reddit_master_collection.at[i, 'created_utc'].date()
    reddit_master_collection.at[i, 'created_utc'] = date
    if i % 20000 == 0:
        print('Iteration', i, end='\r')
        sys.stdout.flush()


       created_utc             author  \
943920  2010-01-01            DesCo83   
943870  2010-01-01           edman007   
943871  2010-01-01                jba   
943872  2010-01-01         krugerlive   
943890  2010-01-01             coasts   
943896  2010-01-01          [deleted]   
943898  2010-01-01             coasts   
943902  2010-01-01            brianoh   
943905  2010-01-01           beerbabe   
943906  2010-01-01            DesCo83   
943869  2010-01-01      SoManyMinutes   
943907  2010-01-01            bmeckel   
943909  2010-01-01           middkidd   
943910  2010-01-01      norwegianrich   
943911  2010-01-01        elsagacious   
943913  2010-01-01            brianoh   
943914  2010-01-01          zwangaman   
943915  2010-01-01           beerbabe   
943916  2010-01-01            DesCo83   
943917  2010-01-01       mrcassavetes   
943918  2010-01-01            brianoh   
943908  2010-01-01          [deleted]   
943868  2010-01-01           edman007   
943867  2010-01-

In [44]:
print(reddit_master_collection.sort_values(by=["created_utc"]))

       created_utc             author  \
943920  2010-01-01            DesCo83   
943870  2010-01-01           edman007   
943871  2010-01-01                jba   
943872  2010-01-01         krugerlive   
943890  2010-01-01             coasts   
943896  2010-01-01          [deleted]   
943898  2010-01-01             coasts   
943902  2010-01-01            brianoh   
943905  2010-01-01           beerbabe   
943906  2010-01-01            DesCo83   
943869  2010-01-01      SoManyMinutes   
943907  2010-01-01            bmeckel   
943909  2010-01-01           middkidd   
943910  2010-01-01      norwegianrich   
943911  2010-01-01        elsagacious   
943913  2010-01-01            brianoh   
943914  2010-01-01          zwangaman   
943915  2010-01-01           beerbabe   
943916  2010-01-01            DesCo83   
943917  2010-01-01       mrcassavetes   
943918  2010-01-01            brianoh   
943908  2010-01-01          [deleted]   
943868  2010-01-01           edman007   
943867  2010-01-

In [106]:
# here is the work for showing the change over time by subreddit/city

years = ['2011', '2012', '2013', '2014', '2015']
subreddits = ["nyc", "losangeles", "chicago", "houston", "phoenix"]

bar_chart_source = reddit_master_collection

frames_by_subreddit = []
for subreddit in subreddits:
    frames_by_subreddit.append(bar_chart_source[bar_chart_source.subreddit == subreddit])


In [80]:
# we need only the year now
bokeh_fixed_year_frame = bar_chart_source
for i, row in bokeh_fixed_year_frame.iterrows():
    date = str(bokeh_fixed_year_frame.at[i, 'created_utc']).split('-')[0]
    bokeh_fixed_year_frame.at[i, 'created_utc'] = date
    if i % 20000 == 0:
        print('Iteration', i, end='\r')
        sys.stdout.flush()

In [144]:
# this should be reusable foir all the bar graphs i'm going to make

bokeh_years = ['2010', '2011', '2012', '2013', '2014']

def showBarGraphByYear(bokeh_source, height, title, xaxis, yaxis, filename):
    source = ColumnDataSource(data=bokeh_source) 

    p = figure(x_range=bokeh_years, y_range=(0, height), plot_height=400, title=title,
               toolbar_location=None, tools="")

    p.vbar(x=dodge('years', -0.25, range=p.x_range), top='Negative', width=0.2, source=source,
           color="#ff4d4d", legend=value("Negative"))

    p.vbar(x=dodge('years',  0.0,  range=p.x_range), top='Neutral', width=0.2, source=source,
           color="#80bfff", legend=value("Neutral"))

    p.vbar(x=dodge('years',  0.25, range=p.x_range), top='Positive', width=0.2, source=source,
           color="#00cc44", legend=value("Positive"))

    p.x_range.range_padding = 0.1
    p.xgrid.grid_line_color = None
    p.legend.location = "top_left"
    p.legend.orientation = "horizontal"
    if not xaxis is None:
        p.xaxis.axis_label = xaxis
    if not yaxis is None:
        p.yaxis.axis_label = yaxis
    
    p.output_backend = "svg"
    export_svgs(p, filename=filename)
    
    show(p)

In [145]:
# bar graph by year - all subreddit content

height = 250
title = "Emotional Rating of Reddit Content by Year"
xaxis = "'Reddit Content' is aggregated from subreddits: nyc, losangeles, chicago, houston, phoenix"
yaxis = "Total number of posts (in thousands)"

negative = []
neutral = []
positive = []
for year in bokeh_years:
    total_negative = 0
    total_neutral = 0
    total_positive = 0
    emotions = bokeh_fixed_year_frame[bokeh_fixed_year_frame.created_utc == year]['emotion']
    for emotion in emotions:
        if emotion == 0:
            total_negative += 1
        if emotion == 2:
            total_neutral += 1
        else:
            total_positive += 1
    negative.append(total_negative/1000)
    neutral.append(total_neutral/1000)
    positive.append(total_positive/1000)


# this is overall, not by subreddit
by_year_source = { 'years': bokeh_years,
               'Negative': negative,
               'Neutral': neutral,
               'Positive': positive}

showBarGraphByYear(by_year_source, height, title, xaxis, yaxis, "reddit_emotions_by_year.svg")

In [168]:
# bar graph by year by subreddit

frames_by_subreddit = []
for subreddit in subreddits:
    frames_by_subreddit.append(bokeh_fixed_year_frame[bokeh_fixed_year_frame.subreddit == subreddit])

amounts_by_subreddit = []
for subreddit_frame in frames_by_subreddit:
    negative = []
    neutral = []
    positive = []
    for year in bokeh_years:
        total_negative = 0
        total_neutral = 0
        total_positive = 0
        emotions = subreddit_frame[subreddit_frame.created_utc == year]['emotion']
        for emotion in emotions:
            if emotion == 0:
                total_negative += 1
            if emotion == 2:
                total_neutral += 1
            else:
                total_positive += 1
        negative.append(total_negative)
        neutral.append(total_neutral)
        positive.append(total_positive)
    
    current_subreddit = str(subreddit_frame["subreddit"].iloc[0])
    height = max([max(negative), max(neutral), max(positive)]) * 1.2
    title = 'Emotional Rating of %s Subreddit Content by Year' %(current_subreddit)
    xaxis = "'Reddit Content' is an aggregation of submissions and comments"
    yaxis = "Total number of posts"
    
    amounts_by_subreddit.append([current_subreddit, negative, neutral, positive])
    
    by_subreddit_by_year_source = { 'years': bokeh_years,
               'Negative': negative,
               'Neutral': neutral,
               'Positive': positive}
    
    file_name = '%s_emotions_by_year.svg' %(current_subreddit)
    showBarGraphByYear(by_subreddit_by_year_source, height, title, xaxis, yaxis, file_name)

In [165]:
def calculateChange(old, new):
    return round((((new - old)/old)*100),4)

In [186]:
# calculate percentage in change of U.S. population -- from census data

census_data_file = "D:\social_media_analytics\\reddit_content\collection\census_data.csv"

census_df = pandas.read_csv(census_data_file)

census_change_over_time = []
for subreddit in subreddits:
    subreddit_frame = census_df[census_df.subreddit == subreddit]
    previous_year_value = None
    subreddit_change_over_time = []
    for year in bokeh_years:
        if not previous_year_value is None:
            current_year_population = subreddit_frame[year].iloc[0]
            subreddit_change_over_time.append(calculateChange(previous_year_value,current_year_population))
            
        previous_year_value = subreddit_frame[year].iloc[0]
        
    census_change_over_time.append([subreddit, subreddit_change_over_time])

print(census_change_over_time)

[['nyc', [-0.0018, 0.188, 1.0086, 0.9074]], ['losangeles', [0.0052, 0.0672, 0.6789, 0.8216]], ['chicago', [0.001, 0.0707, 0.3959, 0.4256]], ['houston', [-0.278, 0.2307, 1.2246, 1.6921]], ['phoenix', [0.0887, 0.1646, 1.4322, 2.0263]]]


In [187]:
# pass in a list of [city,negative yearly count, neuitral, positive]
#def getEmotionChangeOverTimeByYear(source_list):

subreddit_yearly_change = []
for group in amounts_by_subreddit:
    subreddit = group[0]
    negative = group[1]
    neutral = group[2]
    positive = group[3]
    i = 0
    subreddit_negative_change = []
    subreddit_neutral_change = []
    subreddit_positive_change = []
    for year in bokeh_years:
        if not year == '2010':
            subreddit_negative_change.append(calculateChange(negative[i-1], negative[i]))
            subreddit_neutral_change.append(calculateChange(neutral[i-1], neutral[i]))
            subreddit_positive_change.append(calculateChange(positive[i-1], positive[i]))
        i += 1
    subreddit_yearly_change.append([subreddit, subreddit_negative_change])
    subreddit_yearly_change.append([subreddit, subreddit_neutral_change])
    subreddit_yearly_change.append([subreddit, subreddit_positive_change])
print(subreddit_yearly_change)

[['nyc', [96.6203, 9.8966, -0.4498, 0.8712]], ['nyc', [117.177, 11.7263, 3.478, -0.9816]], ['nyc', [99.8447, 7.2211, -0.3283, -3.6972]], ['losangeles', [291.5081, 30.3161, 5.6121, -10.3367]], ['losangeles', [306.0748, 45.5696, 11.8577, -17.1025]], ['losangeles', [274.7439, 32.0106, 5.125, -14.571]], ['chicago', [90.5024, 12.4321, 4.4256, -19.2566]], ['chicago', [114.0766, 22.0936, 17.1478, -21.8095]], ['chicago', [93.6153, 12.0428, 6.278, -22.7984]], ['houston', [201.9693, 58.3485, 10.4839, -21.4121]], ['houston', [201.2945, 57.6799, 24.3869, -20.7558]], ['houston', [184.1717, 56.2139, 13.0797, -21.3664]], ['phoenix', [5430.0, 281.9168, 72.3485, 4.5604]], ['phoenix', [4500.0, 339.1304, 47.2772, -1.3445]], ['phoenix', [5391.1111, 269.7693, 63.9378, 4.3661]]]
